In [2]:
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

import requests
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
gpt_turbo = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [4]:
url = 'https://public-api.meteofrance.fr/public/DPVigilance/v1/textesvigilance/encours'
headers = {
    'accept': '*/*',
    'apikey': os.environ['VIGILENCE_API_KEY']
}
response = requests.get(url, headers=headers)
data = response.json()

In [5]:
def extract_text_items(data):
    text_items = []
    infos = [
        region for region in data['product']['text_bloc_items'] 
        if len(region['bloc_items']) != 0 and region['bloc_id']=='BULLETIN_DEPARTEMENTAL'
    ]
    for info in infos:
        for bloc_item in info.get('bloc_items', []):
            for text_item in bloc_item.get('text_items', []):
                for subdivision in text_item.get('term_items', []):
                    for subdivision_text in subdivision.get('subdivision_text', []):
                        text_items.append("".join(subdivision_text.get('text', '')))
    return text_items

In [6]:
infos = list(set(extract_text_items(data)))

In [7]:
template = """
Write a concise summary in french of the following:
"{informations}"
CONCISE SUMMARY:
"""

In [8]:
prompt = PromptTemplate.from_template(template)


In [9]:
chain = (
    {'informations':RunnablePassthrough()} 
    | prompt 
    | gpt_turbo
)

In [10]:
infos

['Conjonction de grandes marées et de fortes vagues nécessitant une vigilance toute particulière.',
 'Tempête tardive pour la saison, sans caractère exceptionnel, engendrant des vents violents, de fortes vagues et un risque de submersion côtière.',
 'Lundi, "PIERRICK" est accompagnée de vents très forts de secteur Ouest au large de la pointe bretonne.Ces vents engendrent de fortes vagues de Sud-Ouest qui atteignent le littoral nord de la façade atlantique en fin de journée de lundi et sont particulièrement marquées sur l\'ouest du Finistère.De plus, les niveaux d\'eau sont élevés avec des coefficients de marée en hausse (109 à la pleine mer de lundi 18h) et une surélévation du niveau de la mer due aux conditions atmosphériques (surcote).La conjonction de ces très fortes vagues et de ces niveaux d\'eau élevés peut occasionner des submersions marines par franchissement de paquets de mer sur les zones exposées du littoral du Finistère lundi en fin de journée, placé en vigilance "vagues-su

In [11]:
answer = chain.invoke(infos)
print(answer)


Une tempête tardive combinant de fortes marées et des vagues nécessite une vigilance particulière. La dépression nommée "PIERRICK" circule le long des côtes et génère des vents violents et un risque de submersion côtière. Le Finistère est particulièrement touché, avec des vents pouvant atteindre 110 km/h. Les départements de la Manche pourraient également être concernés. Le mardi, la tempête se déplace vers l'Angleterre, provoquant des vents forts et des vagues importantes. Les départements côtiers sont en vigilance pour risque de submersion. 
